In [1]:
NH = 1e22
E = 2.0
dust_model = 1
halo_model = 1
SrcRad = 0.0
th = [10,20,30,40,50]

In [61]:
class Dust:
    def __init__(self, name, a_min, a_max, norm, size_func, halo_model, rho, ZDAtype):
        self.name = name
        self.a_min = a_min
        self.a_max = a_max
        self.norm = norm
        self.size_func = size_func
        self.halo_model = halo_model
        self.rho = rho
        self.ZDAtype = ZDAtype


class Halo:
    import scipy.integrate as integrate
    import numpy as np

    def __init__(self, N_H, E, dusts = None, scatter_model = "RG"):
        self.E = E #AUTUMN: allow for multiple E segments
        self.N_H = N_H
        self.dusts = dusts
        self.scatter_model = scatter_model
        
    # AUTUMN : allow other scatter models
    def dsigma_dOmega(self, a, dust, theta):
        import math
        c = 9.3e-8
        exponent = math.exp(-.4575 * self.E**2 * a**2 * (theta/60)**2)
        F = 1 # AUTUMN: CHANGE 
        return (c * (dust.rho/3)**2 * a**6 * exponent)
        
    def I(self, theta):
        I = 0
        for dust in self.dusts:
            I += self.dust_I(dust, theta)
        return I
    
    def dust_I(self, dust, theta):
        import scipy.integrate as integrate
        def integrand(a):
            return dust.size_func(a) * self.dsigma_dOmega(a, dust, theta)
        integration = integrate.quad(integrand, dust.a_min, dust.a_max)
        return self.E * self.N_H * integration[0]

In [66]:
E = 2.0
N_H = 1e22

def simple_size_func(a):
    return a**(-3.5)

silicate = Dust(name = "silicate", a_min=0.0050, a_max=0.250, norm=1, size_func=simple_size_func, halo_model="RG", rho=3.3, ZDAtype=-1)
graphite = Dust(name = "graphite", a_min=0.0050, a_max=0.250, norm=1, size_func=simple_size_func, halo_model="RG", rho=2.2, ZDAtype=-1)
dusts = [silicate,graphite]

halo = Halo(N_H, E,dusts)

In [67]:
halo.dsigma_dOmega(.005, silicate, 10)

1.758279015518998e-21

In [68]:
halo.dust_I(silicate, 10)

5013510114389.896

In [65]:
halo.I(10)

7241736831896.518